In [2]:
# change directory location if needed:    
data_dir = "/scratch/new_courier_pilot/" ## pilot data directory
# data_dir = "/scratch/EFRCourier_reports/" ## hospital data directory
time_window=10000 # you can chose how long do you want to visualize the data
exp = 'EFRCourierOpenLoop'
subject = 'R1620J' 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import argparse
import glob
import cmlreaders as cml
import json
from matplotlib.ticker import FuncFormatter
import warnings
import numpy as np
import pandas as pd

# Get the full data index
whole_df = cml.CMLReader.get_data_index()

# Define the experiment
exp = 'EFRCourierOpenLoop'

# Get unique subjects for the experiment
subjects = whole_df.query('experiment == @exp')['subject'].unique()

# Initialize results storage
results = []

# Loop through each subject
for subject in subjects:
    print(f"Processing subject: {subject}")
    
    # Get subject data
    sub_df = whole_df.query('experiment == @exp and subject == @subject')
    
    # Get all sessions for this subject
    sessions = sub_df['session'].unique()
    
    # Loop through each session
    for session in sessions:
        print(f"  Processing session: {session}")
        
        try:
            # Create reader for this subject and session
            reader = cml.CMLReader(subject, exp, session=session)
            evs = reader.load('task_events')
            
            # Add the inside_stimuli variable
            evs['inside_stimuli'] = -999
            stimuli_indices = evs[evs['type'] == 'STIM'].index
            for i in stimuli_indices:
                current_offset = evs.loc[i, 'eegoffset']
                
                # Keep checking subsequent rows until difference >= 3000
                j = i + 1
                rows_to_assign = []  # Store indices of rows to assign
                
                while j < len(evs):
                    next_offset = evs.loc[j, 'eegoffset']
                    time_diff = abs(next_offset - current_offset)
                    
                    if time_diff < 3000:
                        # Still within 3 seconds, add this row to assignment list
                        rows_to_assign.append(j)
                        j += 1
                    else:
                        # Found a row with difference >= 4000, stop checking
                        break
                
                # Assign burst_freq to all rows that were within 4000ms of the STIM
                for row_idx in rows_to_assign:
                    evs.loc[row_idx, 'inside_stimuli'] = evs.loc[i, 'stim_params']['burst_freq']
            
            # Filter for intrusions (intrusion == 1)
          
            intrusion_events = evs[(evs['intrusion'] > 0) | (evs['intrusion'] == -1)]
            none_intrusion_events = evs[evs['intrusion'] == 0]

                # Count intrusions for each stimulation condition
            intrusion_count_3hz = (intrusion_events['inside_stimuli'] == 3).sum()
            intrusion_count_8hz = (intrusion_events['inside_stimuli'] == 8).sum()
            intrusion_count_no_stim = (intrusion_events['inside_stimuli'] == -999).sum()
            intrusion_count_total = len(intrusion_events)

                # Count none intrusions for each stimulation condition
            none_intrusion_count_3hz = (none_intrusion_events['inside_stimuli'] == 3).sum()
            none_intrusion_count_8hz = (none_intrusion_events['inside_stimuli'] == 8).sum()
            none_intrusion_count_no_stim = (none_intrusion_events['inside_stimuli'] == -999).sum()
            none_intrusion_count_total = len(none_intrusion_events)

                # Store results
            
            session_result = {
                        'subject': subject,
                        'session': session,
                        'intrusion_count_3hz': intrusion_count_3hz,
                        'intrusion_count_8hz': intrusion_count_8hz,
                        'intrusion_count_no_stim': intrusion_count_no_stim,
                        'intrusion_count_total': intrusion_count_total,
                        'none_intrusion_count_3hz': none_intrusion_count_3hz,
                        'none_intrusion_count_8hz': none_intrusion_count_8hz,
                        'none_intrusion_count_no_stim': none_intrusion_count_no_stim,
                        'none_intrusion_count_total': none_intrusion_count_total,
                        'relative_rate_3hz': intrusion_count_3hz / none_intrusion_count_3hz if none_intrusion_count_3hz != 0 else None,
                        'relative_rate_8hz': intrusion_count_8hz / none_intrusion_count_8hz if none_intrusion_count_8hz != 0 else None,
                        'relative_rate_no_stim': intrusion_count_no_stim / none_intrusion_count_no_stim if none_intrusion_count_no_stim != 0 else None
                }
            results.append(session_result)
            
            print(f"    Intrusions - 3Hz: {intrusion_count_3hz}, 8Hz: {intrusion_count_8hz}, No stim: {intrusion_count_no_stim}, Total: {intrusion_count_total}")
            
        except Exception as e:
            print(f"    Error processing session {session}: {str(e)}")
            continue

# Convert results to DataFrame
results_df = pd.DataFrame(results)

# Display summary statistics
print("\n" + "="*50)
print("INTRUSION ANALYSIS SUMMARY")
print("="*50)

if len(results_df) > 0:
    print(f"Total subjects processed: {results_df['subject'].nunique()}")
    print(f"Total sessions processed: {len(results_df)}")
    print(f"\nIntrusion statistics across all sessions:")
    print(f"Mean intrusions per session:")
    print(f"  3Hz stimulation: {results_df['intrusion_count_3hz'].mean():.3f} ± {results_df['intrusion_count_3hz'].std():.3f}")
    print(f"  8Hz stimulation: {results_df['intrusion_count_8hz'].mean():.3f} ± {results_df['intrusion_count_8hz'].std():.3f}")
    print(f"  No stimulation: {results_df['intrusion_count_no_stim'].mean():.3f} ± {results_df['intrusion_count_no_stim'].std():.3f}")
    print(f"  Total: {results_df['intrusion_count_total'].mean():.3f} ± {results_df['intrusion_count_total'].std():.3f}")
    print(f"\nTotal intrusions across all sessions:")
    print(f"  3Hz stimulation: {results_df['intrusion_count_3hz'].sum()}")
    print(f"  8Hz stimulation: {results_df['intrusion_count_8hz'].sum()}")
    print(f"  No stimulation: {results_df['intrusion_count_no_stim'].sum()}")
    print(f"  Total: {results_df['intrusion_count_total'].sum()}")
    
    # Display the results DataFrame
    print(f"\nDetailed results:")
    print(results_df.to_string(index=False))
else:
    print("No results to display. Check for errors in processing.")
   

Processing subject: FBG490
  Processing session: 0
    Intrusions - 3Hz: 7, 8Hz: 2, No stim: 32, Total: 41
  Processing session: 1
    Intrusions - 3Hz: 4, 8Hz: 4, No stim: 53, Total: 61
  Processing session: 2
    Intrusions - 3Hz: 1, 8Hz: 4, No stim: 58, Total: 63
Processing subject: FBG491
  Processing session: 1
    Intrusions - 3Hz: 6, 8Hz: 4, No stim: 20, Total: 30
  Processing session: 2
    Intrusions - 3Hz: 4, 8Hz: 2, No stim: 19, Total: 25
Processing subject: FR491
  Processing session: 1
    Error processing session 1: Unable to find the requested file in any of the expected locations:
 /protocols/pyfr/subjects/FR491/experiments/EFRCourierOpenLoop/sessions/1/behavioral/current_processed/task_events.json
/data/events/pyFR/FR491_None_events.mat
  Processing session: 2
    Error processing session 2: Unable to find the requested file in any of the expected locations:
 /protocols/pyfr/subjects/FR491/experiments/EFRCourierOpenLoop/sessions/2/behavioral/current_processed/task_even

In [10]:
intrusion_events[['intrusion', 'type','item']]

,intrusion,type,item
110,-1,REC_WORD,BOLTS
113,-1,REC_WORD,PANTS
114,-1,REC_WORD_VV,<>
116,-1,REC_WORD_VV,<>
117,-1,REC_WORD,CANOLLI
128,-1,REC_WORD_VV,<>
129,-1,REC_WORD_VV,<>
131,-1,REC_WORD_VV,<>
132,-1,REC_WORD_VV,<>
135,-1,REC_WORD_VV,<>
